# Plot `TNCode` and operators on `TNCode`

In [2]:
using LightGraphs, GraphPlot, ITensors 
using NBInclude
@nbinclude("TN_code_types.ipynb")





"""
    edge_to_graph_edge(edge::Set{Int64},n_virtual::Int64)

Because `LightGraph` indices start at 1, but `CodeGraph` indices
include negative numbers, this converts `CodeGraph` edges to have
positive values.
"""
function edge_to_graph_edge(edge::Set{Int64},n_virtual::Int64)
    
    output = [edge...] .+ n_virtual
    for α in 1:2
        if output[α] < n_virtual
            output[α] += 1
        end
    end
        
    return output
end





"""
    new_graph(code_graph::CodeGraph)

Given a `CodeGraph`, returns a `LightGraph` for plotting.
"""
function new_graph(code_graph::CodeGraph)
    
    Nodes = nodes(code_graph)
    virtual_nodes = filter(x->x<0,Nodes)
    n_virtual = length(virtual_nodes)
    
    Edges = edges(code_graph)
    
    output = Graph(length(Nodes))
    for edge in Edges
        graph_edge = edge_to_graph_edge(edge,n_virtual)
        add_edge!(output,graph_edge...)
    end
    
    return output
end


new_graph(code::TNCode) = new_graph(code.code_graph)






"""
    code_plot(code::TNCode)

Plots a `TNcode`, with physical qubit nodes coloured red and virtual
tensors coloured green.
"""
function code_plot(code::TNCode;use_coords=true)
    
    Nodes = nodes(code.code_graph)
    graph = new_graph(code) 

    
    nodelabels = [(node_types(code,node) == "physical") ? node : " "
        for node in Nodes]  
    
    nodecolours = [(node_types(code,node) == "physical") ?
        "tomato" : "lightgreen" 
        for node in Nodes]
    
    
    locs_x = Float64[]
    locs_y = Float64[]
    for node in Nodes         
        locs = coords(code,node)
        push!(locs_x,locs[1])
        push!(locs_y,locs[2])
    end

    if use_coords
        display(gplot(graph,locs_x,locs_y,nodefillc=nodecolours,nodelabel=nodelabels))
    else
        display(gplot(graph,nodefillc=nodecolours,nodelabel=nodelabels))
    end    
end





"""
    operator_plot(
        code::TNCode,
        operator::Array{Int64,1};
        use_coords = true)

Plots `TNCode` operator(s).
"""
function operator_plot(
        code::TNCode,
        operator::Array{Int64,1};
        use_coords = true)
    
    Nodes = nodes(code.code_graph)
    graph = new_graph(code) 

    
    nodelabels = []
    nodecolours = String[]
    for node in Nodes
        node_type = node_types(code,node)
        
        if node_type == "physical"
            pauli = pauli_rep_change(operator[node])
            push!(nodelabels,pauli)
            if pauli == 'I'
                push!(nodecolours,"goldenrod1")
            else
                push!(nodecolours,"tomato")
            end
        else
            push!(nodelabels," ")
            push!(nodecolours,"lightgreen")
        end
    end
        
    
    locs_x = Float64[]
    locs_y = Float64[]
    for node in Nodes         
        locs = coords(code,node)
        push!(locs_x,locs[1])
        push!(locs_y,locs[2])
    end
    
    
    if use_coords
        display(gplot(graph,locs_x,locs_y,nodefillc=nodecolours,nodelabel=nodelabels))
    else
        display(gplot(graph,nodefillc=nodecolours,nodelabel=nodelabels))
    end   
    
end



operator_plot(
    code::TNCode,
    operators::Array{Array{Int64,1},1};
    use_coords = true) = 
operator_plot.(Ref(code),operators;use_coords)

shift_coords!

## Testing

In [7]:
# using NBInclude
# @nbinclude("Code_functions.ipynb")


# b = @allocated TN_five_qubit = TNCode(five_qubit)

# code_plot(TN_five_qubit)

In [6]:
# operator_plot(TN_five_qubit,TN_five_qubit.stabilizers)